In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 68.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load product data
df = pd.read_csv("cleaned_amazon_products.csv")

In [ ]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
import pickle

# Load product data
df = pd.read_csv("cleaned_amazon_products.csv")

# Normalize price and ratings to a scale (0-1)
max_price = df["actual_price"].max()
normalized_price = df["actual_price"] / max_price

max_rating = df["ratings"].max()  # Assume ratings are numerical
normalized_rating = df["ratings"] / max_rating  # Scale ratings between 0-1

# Load pre-trained sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for product names
product_embeddings = model.encode(df["name"].tolist(), convert_to_tensor=True)

# Convert embeddings to NumPy
product_embeddings = product_embeddings.cpu().numpy()

# Append price and rating as extra features
price_embeddings = normalized_price.values.reshape(-1, 1)  # Reshape to match dimensions
rating_embeddings = normalized_rating.values.reshape(-1, 1)  # Reshape to match dimensions

# Combine text embeddings with price & rating
combined_embeddings = np.hstack((product_embeddings, price_embeddings, rating_embeddings))

# Create FAISS index
index = faiss.IndexFlatL2(combined_embeddings.shape[1])
index.add(combined_embeddings)

# Save index and data
with open("embeddings_faiss.pkl", "wb") as f:
    pickle.dump((df, combined_embeddings), f)

print("✅ FAISS Index with price & rating saved!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS Index with price & rating saved!
